In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
%matplotlib inline
import matplotlib.pyplot as plt

import sys 
import inspect
import seaborn as sns
import os

import perturbseq as perturb

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, color_map='viridis')
sc.logging.print_versions()

scanpy==1.5.1 anndata==0.7.1 umap==0.4.4 numpy==1.17.2 scipy==1.4.1 pandas==0.24.2 scikit-learn==0.22 statsmodels==0.11.0 python-igraph==0.7.1 louvain==0.6.1 leidenalg==0.8.1


In [2]:
PROTEIN='TP53'
DATA='/ahg/regevdata/projects/bn10_cancer_variants/bn10_oana_2018-06-22/results/2018-06-22/results/2020-02-14'
SUBSAMPLED=DATA+'/'+PROTEIN+'/subsampled/A549.'+PROTEIN

counts=20000

FIGS=DATA+'/2020-09-15_Figures/'
os.system('mkdir -p '+FIGS)
print(FIGS)

/ahg/regevdata/projects/bn10_cancer_variants/bn10_oana_2018-06-22/results/2018-06-22/results/2020-02-14/2020-09-15_Figures/


In [3]:
adata=sc.read(SUBSAMPLED+'.channel_1to32.counts_per_cell'+str(counts)+'.regress_out_vbc.analysis.1000cells.new_format.h5ad',cache=False)
adata

Only considering the two last: ['.new_format', '.h5ad'].
Only considering the two last: ['.new_format', '.h5ad'].


AnnData object with n_obs × n_vars = 85778 × 1224 
    obs: 'batch', 'known', 'n_counts', 'n_genes', 'percent_mito', 'n_counts_downsampled', 'n_counts_original', 'uninfected', 'vbc.counts', 'batch_num18', 'batch_num8', 'batch_num25', 'batch_num22', 'batch_num24', 'batch_num28', 'batch_num27', 'batch_num23', 'batch_num17', 'batch_num19', 'batch_num12', 'batch_num31', 'batch_num5', 'batch_num15', 'batch_num7', 'batch_num9', 'batch_num4', 'batch_num2', 'batch_num13', 'batch_num6', 'batch_num26', 'batch_num16', 'batch_num10', 'batch_num1', 'batch_num29', 'batch_num0', 'batch_num11', 'batch_num30', 'batch_num3', 'batch_num14', 'batch_num21', 'batch_num20', 'louvain', 'G1.S', 'G2.M', 'M', 'M.G1', 'S', 'phase.multi', 'guide', 'guide.compact', 'G105C', 'G266E', 'G154V', 'Y220C', 'V216V', 'WT', 'M246V', 'E286K', 'C176Y', 'S366A', 'H193Y', 'L344L', 'E339K', 'P47P', 'V173M', 'V272L', 'R158H', 'C135F', 'A83V', 'L194R', 'T312S', 'S127F', 'R249S', 'R337L', 'R273H', 'V272M', 'R273L', 'Q5R', 'R249M', 

In [10]:
#table
PROTEIN='TP53'
OUT='/ahg/regevdata/projects/bn10_cancer_variants/bn10_oana_2018-06-22/results/2018-06-22/results/2020-02-14/'+PROTEIN+'/anno'
#original anno
anno=pd.read_csv(OUT+'/'+PROTEIN+'.anno.complete.csv',sep='\t')
anno.index=anno['Variant']
print(anno.head())

        Variant  Position From To mutation_type  Count_pancan  Count_ExAC  \
Variant                                                                     
R175H     R175H     175.0    C  T        Single         993.0         NaN   
R273C     R273C     273.0    G  A        Single         677.0         NaN   
R248Q     R248Q     248.0    C  T        Single         656.0         NaN   
R273H     R273H     273.0    C  T        Single         622.0         NaN   
R248W     R248W     248.0    G  A        Single         550.0         NaN   

        control_status control_color Library_synthesis   long_name  
Variant                                                             
R175H          unknown         white                in  TP53_R175H  
R273C          unknown         white                in  TP53_R273C  
R248Q          unknown         white    not in library         NaN  
R273H          unknown         white                in  TP53_R273H  
R248W          unknown         white          

In [11]:
anno['control_status2']=list(anno['control_status'])
for i in range(anno.shape[0]):
    var=anno.index[i]
    
    if var in ['unassigned','WT']:
        continue
        
    pos=str(int(anno['Position'][i]))
    
    
    if pos in ['175','248','273']:
        anno['control_status2'][i]='hotspot'
anno.loc['unassigned','control_status2']='unknown'

/ahg/regevdata/users/oursu/software/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [12]:
anno.loc['unassigned','control_color']='slateblue'

anno['text_color']='black'

anno.loc[anno['control_status']=='ExAC','text_color']='gray'
anno.loc[anno['control_status']=='synonymous','text_color']='gray'
anno.loc[anno['control_status2']=='hotspot','text_color']='blue'

anno.loc[anno['control_status']=='ExAC','control_color']='lightgray'
anno.loc[anno['control_status']=='synonymous','control_color']='gray'



In [13]:
anno.head()

,Variant,Position,From,To,mutation_type,Count_pancan,Count_ExAC,control_status,control_color,Library_synthesis,long_name,control_status2,text_color
Variant,,,,,,,,,,,,,
R175H,R175H,175.0,C,T,Single,993.0,NaN,unknown,white,in,TP53_R175H,hotspot,blue
R273C,R273C,273.0,G,A,Single,677.0,NaN,unknown,white,in,TP53_R273C,hotspot,blue
R248Q,R248Q,248.0,C,T,Single,656.0,NaN,unknown,white,not in library,NaN,hotspot,blue
R273H,R273H,273.0,C,T,Single,622.0,NaN,unknown,white,in,TP53_R273H,hotspot,blue
R248W,R248W,248.0,G,A,Single,550.0,NaN,unknown,white,in,TP53_R248W,hotspot,blue


In [14]:
anno['control_status2'].value_counts()

unknown       69
synonymous    16
ExAC          10
hotspot        7
Name: control_status2, dtype: int64

In [15]:
anno['text_color'].value_counts()

black    69
gray     26
blue      7
Name: text_color, dtype: int64

In [16]:
anno['control_color'].value_counts()

white        75
gray         16
lightgray    10
slateblue     1
Name: control_color, dtype: int64

In [ ]:
#control colors


In [17]:
anno.to_csv(OUT+'/'+PROTEIN+'.anno.complete2.csv',sep='\t')